請記得要先將tensorflow更新到最新的版本

pip install --upgrade  tensorflow

pip install --upgrade  tensorflow-gpu

之後再將numpy更新到最新的版本

pip install --upgrade  numpy


如果第一個cell有numpy的錯誤，請將 numpy unsatll後再安裝最新的版本


這是用我們train好的模型，感謝王俊凱大大。模型編號1559725764 在第一個cell的第35行設定


In [15]:
#接下來串接
import tensorflow as tf
import numpy as np
import os, argparse, time, random
from model import BiLSTM_CRF
from utils import str2bool, get_logger, get_entity
from data import read_corpus, read_dictionary, tag2label, random_embedding, vocab_build

## Session configuration
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # default: 0
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.2  # need ~700MB GPU memory

parser = argparse.ArgumentParser(description='BiLSTM-CRF for Chinese NER task')
#parser.add_argument('--update_train_data', type=str, default='data_path', help='update training data set')

parser.add_argument('--train_data', type=str, default='data_path', help='train data source')
parser.add_argument('--test_data', type=str, default='data_path', help='test data source')
parser.add_argument('--batch_size', type=int, default=64, help='#sample of each minibatch')
parser.add_argument('--epoch', type=int, default=40, help='#epoch of training')#default=40

parser.add_argument('--hidden_dim', type=int, default=300, help='#dim of hidden state')
parser.add_argument('--optimizer', type=str, default='Adam', help='Adam/Adadelta/Adagrad/RMSProp/Momentum/SGD')
parser.add_argument('--CRF', type=str2bool, default=True, help='use CRF at the top layer. if False, use Softmax')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--clip', type=float, default=5.0, help='gradient clipping')
parser.add_argument('--dropout', type=float, default=0.5, help='dropout keep_prob')
parser.add_argument('--update_embedding', type=str2bool, default=True, help='update embedding during training')
parser.add_argument('--pretrain_embedding', type=str, default='random', help='use pretrained char embedding or init it randomly')
parser.add_argument('--embedding_dim', type=int, default=300, help='random init char embedding_dim')
parser.add_argument('--shuffle', type=str2bool, default=True, help='shuffle training data before each epoch')
parser.add_argument('--mode', type=str, default='demo', help='train/test/demo/update')
parser.add_argument('--demo_model', type=str, default='1559725764', help='model for test and demo')
args = parser.parse_args(args=[])

## get char embeddings
word2id = read_dictionary(os.path.join('.', args.train_data, 'word2id.pkl'))
if args.pretrain_embedding == 'random':
    embeddings = random_embedding(word2id, args.embedding_dim)
else:
    embedding_path = 'pretrain_embedding.npy'
    embeddings = np.array(np.load(embedding_path), dtype='float32')


train_path = os.path.join('.', args.train_data, 'train_data')
test_path = os.path.join('.', args.test_data, 'test_data')
train_data = read_corpus(train_path)
test_data = read_corpus(test_path); test_size = len(test_data)

paths = {}
timestamp = str(int(time.time())) if args.mode == 'train' else args.demo_model
output_path = os.path.join('.', args.train_data+"_save", timestamp)

if not os.path.exists(output_path): os.makedirs(output_path)
    
summary_path = os.path.join(output_path, "summaries")
paths['summary_path'] = summary_path

if not os.path.exists(summary_path): os.makedirs(summary_path)
model_path = os.path.join(output_path, "checkpoints/")
if not os.path.exists(model_path): os.makedirs(model_path)
ckpt_prefix = os.path.join(model_path, "model")
paths['model_path'] = ckpt_prefix

result_path = os.path.join(output_path, "results")
paths['result_path'] = result_path

if not os.path.exists(result_path): os.makedirs(result_path)
log_path = os.path.join(result_path, "log.txt")
paths['log_path'] = log_path

get_logger(log_path).info(str(args))

print(paths['result_path'])

vocab_size: 4271


Namespace(CRF=True, batch_size=64, clip=5.0, demo_model='1559725764', dropout=0.5, embedding_dim=300, epoch=40, hidden_dim=300, lr=0.001, mode='demo', optimizer='Adam', pretrain_embedding='random', shuffle=True, test_data='data_path', train_data='data_path', update_embedding=True)


.\data_path_save\1559725764\results


In [16]:
#讀取該模型的參數
ckpt_file = tf.train.latest_checkpoint(model_path)
print(ckpt_file)
paths['model_path'] = ckpt_file
#build_graph 建構一次就可以了
tf.reset_default_graph()#如果沒有家這個，會有kernel is already exist的錯誤
model = BiLSTM_CRF(args, embeddings, tag2label, word2id, paths, config=config)
model.build_graph()

.\data_path_save\1559725764\checkpoints/model-633400


In [17]:
#這邊就放入你們要找的文章內容
article = "鏡週刊報導油商陳世憲去年違反聯合國禁運，偷偷賣油給北韓，成為台灣首位遭資恐防制法制裁的對象，數億元資產全被凍結，但陳甚至致電至法務部，揚言殺光法務部的政次陳明堂及洗錢防制辦公室檢察官蔡佩玲全家，恐嚇危害執法人員生命。台北地檢署今上午分他字案調查，案由為恐嚇。"

In [18]:
print("" )
print("新聞文章內文 : " + article)
print("" )

#下面這段可以重複使用，找出三個entity，看你們要不要把它包成一個function
saver = tf.train.Saver()
with tf.Session(config=config) as sess:
    saver.restore(sess, ckpt_file)
    demo_sent = article.strip()
    demo_data = [(demo_sent, ['O'] * len(demo_sent))]
    tag = model.demo_one(sess, demo_data)
    PER, LOC, ORG = get_entity(tag, demo_sent)
    print("" )
    print("分析結果如下:" )
    print('文章出現人名: {}\n文章出現地名: {}\n文章出現組織: {}'.format(PER, LOC, ORG))


新聞文章內文 : 鏡週刊報導油商陳世憲去年違反聯合國禁運，偷偷賣油給北韓，成為台灣首位遭資恐防制法制裁的對象，數億元資產全被凍結，但陳甚至致電至法務部，揚言殺光法務部的政次陳明堂及洗錢防制辦公室檢察官蔡佩玲全家，恐嚇危害執法人員生命。台北地檢署今上午分他字案調查，案由為恐嚇。

INFO:tensorflow:Restoring parameters from .\data_path_save\1559725764\checkpoints/model-633400


Restoring parameters from .\data_path_save\1559725764\checkpoints/model-633400



分析結果如下:
文章出現人名: ['陳世憲', '陳明堂', '蔡佩玲']
文章出現地名: ['北韓', '台灣']
文章出現組織: ['聯合國', '法務部', '揚言殺光法務部', '洗錢防制辦公室', '台北地檢署']


下面是確定存在check point 的variable的結構跟tensorflow內的結構是否依樣，因為如果你們tensorflow版本汰舊的話會有
NotFoundError: ..... not found in checkpoint files這類的錯誤，通常是你的tensorflow版本太舊了

In [12]:
#這裡會把你checkpoint中儲存的模型內的參數結構顯示出來
import os
from tensorflow.python import pywrap_tensorflow

reader = pywrap_tensorflow.NewCheckpointReader(ckpt_file)
var_to_shape_map = reader.get_variable_to_shape_map()
# Print tensor name and values
for key in var_to_shape_map:
    print("tensor_name: ", key)

tensor_name:  bi-lstm/bidirectional_rnn/bw/lstm_cell/kernel
tensor_name:  bi-lstm/bidirectional_rnn/bw/lstm_cell/bias
tensor_name:  train_step/proj/W/Adam
tensor_name:  bi-lstm/bidirectional_rnn/fw/lstm_cell/kernel
tensor_name:  bi-lstm/bidirectional_rnn/fw/lstm_cell/bias
tensor_name:  train_step/bi-lstm/bidirectional_rnn/bw/lstm_cell/kernel/Adam
tensor_name:  train_step/bi-lstm/bidirectional_rnn/bw/lstm_cell/bias/Adam
tensor_name:  proj/W
tensor_name:  train_step/words/_word_embeddings/Adam_1
tensor_name:  proj/b
tensor_name:  train_step/bi-lstm/bidirectional_rnn/bw/lstm_cell/kernel/Adam_1
tensor_name:  train_step/bi-lstm/bidirectional_rnn/bw/lstm_cell/bias/Adam_1
tensor_name:  train_step/beta1_power
tensor_name:  train_step/beta2_power
tensor_name:  train_step/bi-lstm/bidirectional_rnn/fw/lstm_cell/kernel/Adam
tensor_name:  train_step/bi-lstm/bidirectional_rnn/fw/lstm_cell/bias/Adam
tensor_name:  train_step/bi-lstm/bidirectional_rnn/fw/lstm_cell/kernel/Adam_1
tensor_name:  train_step

In [13]:
tf.global_variables()#這邊則是tensorflow內部的結構，妳可以比對看看是否跟上面的一樣

[<tf.Variable 'words/_word_embeddings:0' shape=(4271, 300) dtype=float32_ref>,
 <tf.Variable 'bi-lstm/bidirectional_rnn/fw/lstm_cell/kernel:0' shape=(600, 1200) dtype=float32_ref>,
 <tf.Variable 'bi-lstm/bidirectional_rnn/fw/lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>,
 <tf.Variable 'bi-lstm/bidirectional_rnn/bw/lstm_cell/kernel:0' shape=(600, 1200) dtype=float32_ref>,
 <tf.Variable 'bi-lstm/bidirectional_rnn/bw/lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>,
 <tf.Variable 'proj/W:0' shape=(600, 7) dtype=float32_ref>,
 <tf.Variable 'proj/b:0' shape=(7,) dtype=float32_ref>,
 <tf.Variable 'transitions:0' shape=(7, 7) dtype=float32_ref>,
 <tf.Variable 'train_step/global_step:0' shape=() dtype=int32_ref>,
 <tf.Variable 'train_step/beta1_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'train_step/beta2_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'train_step/words/_word_embeddings/Adam:0' shape=(4271, 300) dtype=float32_ref>,
 <tf.Variable 'train_step/words/_word_embed